# Deep Dive into Spark (Python)
 1. Spark Shell
 2. Parallel Programming
 3. context
 4. RDD
 5. Transformations
 6. Programming with RDD

## Import  Library

In [1]:
import findspark
findspark.init()
from pyspark import SparkConf, SparkContext 
from pyspark.sql import SparkSession

## Setup spark context

In [2]:
conf = SparkConf().setAppName('hello').setMaster('local')
sc = SparkContext(conf=conf)

## Paralize Example

In [3]:
words = sc.parallelize (
   ["scala", 
   "java", 
   "hadoop", 
   "spark", 
   "akka",
   "spark vs hadoop", 
   "pyspark",
   "pyspark and spark"]
)
counts = words.count()
counts

8

## RDD Example

In [4]:
pathFile = "wine-data.csv"
rdd = sc.textFile(pathFile)

In [5]:
rdd.take(1)

[';country;description;designation;points;price;province;region_1;region_2;taster_name;taster_twitter_handle;title;variety;winery']

### 1.Count wine data point > 90

In [24]:
def mapFloat(stringData):
    stringData = stringData.split(";")
    try:
        data = float(stringData[5])
    except:
        data = 0.0
        stringData[5]=str(data)
    return stringData

rddHeaderRemoved = rdd.filter(lambda lines: lines.split(";")[1] != "country")
rddLowerCase = rddHeaderRemoved.map(lambda sentences: sentences.lower())
rddLowerCase.filter(lambda data: data.split(";")[5] != "" ). \
    filter(lambda data: data.split(";")[5]).map(mapFloat).filter(lambda data : float(data[5]) > 90).count()

4495

### 2.Most grape type for make wine

In [81]:
rddLowerCase.map(lambda data: (data.split(";")[12],1)).groupByKey(). \
    mapValues(sum).sortBy(lambda value: value[1],ascending= False). \
    take(5)

[('pinot noir', 12978),
 ('chardonnay', 11513),
 ('cabernet sauvignon', 9180),
 ('red blend', 8685),
 ('bordeaux-style red blend', 6792)]

### 3.Most grape type and provienci for make wine

In [84]:
from operator import add
rddLowerCase.map(lambda data: (data.split(";")[12]+","+data.split(";")[6],1)). \
    groupByKey().mapValues(sum).sortBy(lambda value: value[1],ascending= False). \
    filter(lambda value: value[0] != ",").take(5)

[('pinot noir,california', 6824),
 ('cabernet sauvignon,california', 5591),
 ('chardonnay,california', 5142),
 ('bordeaux-style red blend,bordeaux', 4547),
 ('pinot noir,oregon', 2657)]

# 5 most expensive price wine with provience

In [111]:
def mapFloat(stringData):
    stringData = list(stringData)
    try:
        data = float(stringData[0])
    except:
        data = 0.0
        stringData[0]=str(data)
    return stringData

listData = rddLowerCase.map(lambda data: data.split(";")). \
    map(lambda data: (data[5],data[6],data[12])). \
    filter(lambda value: value[0] != "").map(mapFloat). \
    sortBy(lambda value: float(value[0]),ascending= False).take(5)
for data in listData:
    print(data)

['3300.0', 'bordeaux', 'bordeaux-style red blend']
['2500.0', 'bordeaux', 'bordeaux-style red blend']
['2500.0', 'burgundy', 'pinot noir']
['2013.0', 'california', 'chardonnay']
['2000.0', 'bordeaux', 'bordeaux-style red blend']


[1, 'd']